In [1]:
# 输入
import tensorflow as tf
import numpy as np
import math

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
seq_length = 25
num_class = 1000
input_x = tf.placeholder(tf.int32,[None,seq_length],name='input_x')
input_y = tf.placeholder(tf.float32,[None,num_class],name='input_y')
keep_prob=tf.placeholder(tf.float32,name='keep_prob')

In [3]:
print(input_x)
print(input_y)

Tensor("input_x:0", shape=(?, 25), dtype=int32)
Tensor("input_y:0", shape=(?, 1000), dtype=float32)


#嵌入层

In [4]:
vacab_size = 140000
embedding_dim = 200
embedding = tf.get_variable('embedding',[vacab_size,embedding_dim])
embedding_inputs = tf.nn.embedding_lookup(embedding,input_x)

embedding_inputs_expend = tf.expand_dims(embedding_inputs,-1)


print(embedding)
print(embedding_inputs)
print(embedding_inputs_expend)

<tf.Variable 'embedding:0' shape=(140000, 200) dtype=float32_ref>
Tensor("embedding_lookup:0", shape=(?, 25, 200), dtype=float32)
Tensor("ExpandDims:0", shape=(?, 25, 200, 1), dtype=float32)


In [5]:
# cnn pooling
filter_sizes = [3,4,5]
filter_nums = 2
pooled_outputs = []
for filter_size in filter_sizes:
    filter_shape = [filter_size,embedding_dim,1,filter_nums]

    W = tf.Variable(tf.truncated_normal(filter_shape,stddev=0.1),name='W')
    b = tf.Variable(tf.constant(0.1,shape=[filter_nums]),name='b')

    conv = tf.nn.conv2d(input=embedding_inputs_expend,filter=W,strides=[1,1,1,1],padding='VALID',name='conv')
    print(conv)

    h = tf.nn.relu(tf.nn.bias_add(conv,b),name = 'relu')
    
    pooled = tf.nn.max_pool(h,ksize=[1,seq_length-filter_size+1,1,1],strides=[1,1,1,1],padding='VALID',name='pool')
    pooled_outputs.append(pooled)
print(pooled_outputs)

Tensor("conv:0", shape=(?, 23, 1, 2), dtype=float32)
Tensor("conv_1:0", shape=(?, 22, 1, 2), dtype=float32)
Tensor("conv_2:0", shape=(?, 21, 1, 2), dtype=float32)
[<tf.Tensor 'pool:0' shape=(?, 1, 1, 2) dtype=float32>, <tf.Tensor 'pool_1:0' shape=(?, 1, 1, 2) dtype=float32>, <tf.Tensor 'pool_2:0' shape=(?, 1, 1, 2) dtype=float32>]


In [6]:
num_filters_totle = filter_nums * len(filter_sizes)
h_pool = tf.concat(pooled_outputs,3)
print(h_pool)

Tensor("concat:0", shape=(?, 1, 1, 6), dtype=float32)


In [7]:
h_pool_flat = tf.reshape(h_pool,[-1,num_filters_totle])
print(h_pool_flat)

Tensor("Reshape:0", shape=(?, 6), dtype=float32)


In [8]:
# 全连接层 计算最终的输出结果
h_drop = tf.nn.dropout(h_pool_flat,keep_prob)

w1 = tf.get_variable('w1',shape=[num_filters_totle,num_class],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.constant(0.1,shape=[num_class]),name='b1')
l2_loss = tf.constant(0.0)
l2_loss += tf.nn.l2_loss(w1)
l2_loss += tf.nn.l2_loss(b1)

scores = tf.nn.xw_plus_b(h_drop,w1,b1,name='scores')
print(scores)

Tensor("scores:0", shape=(?, 1000), dtype=float32)


In [9]:
# 损失函数
losses = tf.nn.softmax_cross_entropy_with_logits(logits=scores,labels=input_y)
l2_reg_lambed = 0.0
loss = tf.reduce_mean(losses)+l2_reg_lambed*l2_loss
print(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Tensor("add_2:0", shape=(), dtype=float32)


In [10]:
# accuracy 
print(input_y)
print(scores)

Tensor("input_y:0", shape=(?, 1000), dtype=float32)
Tensor("scores:0", shape=(?, 1000), dtype=float32)


In [11]:
# 获取top_k 的索引，后期通过索引可找到相应的词的位置
top_k = 5
_,input_result_idx = tf.nn.top_k(input_y,top_k)
_,predict_result_idx = tf.nn.top_k(scores,top_k)
print(input_result_idx)
print(predict_result_idx)

Tensor("TopKV2:1", shape=(?, 5), dtype=int32)
Tensor("TopKV2_1:1", shape=(?, 5), dtype=int32)


In [19]:
#有了两个的索引，用来计算acc
batch_size = 1
right_label_num = 0
right_labbel_at_pos_num = tf.Variable(tf.zeros(shape=[top_k]),name='right_labbel_at_pos_num')
sample_num = 0
all_marked_label_num = 0

for bh in range(batch_size):
    sample_num += 1
    # 取出两个预测的值 和实际的值
    input_y = input_result_idx[bh]
    predict_y = predict_result_idx[bh]    
    # 实际的值去重
    input_y,_ = tf.unique(input_y)
   
    # 将两个tensor变成可以迭代的tensor
    input_y = tf.map_fn(lambda x:x, input_y)
    predict_y = tf.map_fn(lambda x:x,predict_y)
    
    print(predict_y)
    for i in range(top_k):
        predict = predict_y[i]
        for j in range(top_k):
            inpt = input_y[i]
            if tf.cast(tf.equal(predict,inpt),dtype=tf.bool):
                right_label_num += 1
                right_labbel_at_pos_num[pos] += 1


precision = 0.0
for pos in range(top_k):
    right_num = right_labbel_at_pos_num[pos]
    precision += ((right_num / float(sample_num))) / math.log(2.0 + pos)
    
recall = float(right_label_num) / all_marked_label_num
acc =  (precision * recall) / (precision + recall)

    
print(input_y[1])

Tensor("map_12/TensorArrayStack/TensorArrayGatherV3:0", shape=(5,), dtype=int32)


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.